# Construct FF States

In [ ]:
import mmf_setup;mmf_setup.nbinit()
%pylab inline --no-import-all
from nbimports import * 
from mmf_hfb import tf_completion as tf
from mmf_hfb.FuldeFerrelState import FFState
import itertools
from mmfutils.plot import imcontourf
tf.MAX_DIVISION = 200
plt.figure(figsize(10,4))
clear_output()

* For a FF state, the ground state should have zero overall current:
$$
n_a (q + \delta q) + n_b(q-\delta q) = 0
$$
which means:
$$
\frac{n_a}{n_b}=\frac{\delta q - q}{\delta q +q}
$$
Let $n_a/n_b=r$, then
$$
\frac{\delta q}{q}=\frac{1 + r}{1-r}=\frac{n_a+n_b}{n_b-n_a}
$$

In [ ]:
delta = 1
mu = 2 * delta
dmu = 0.9
ff = FFState(mu=mu, dmu=dmu, delta=delta, dim=1, k_c=100,fix_g=True, bStateSentinel=True)
assert ff.bSuperfluidity

In [ ]:
dmus = np.linspace(0.85, 1.1, 20) * dmu
fs = [ff.f(mu=mu, dmu=dm, delta=delta) for dm in dmus]
plt.plot(dmus, fs)
plt.axvline(dmu)

In [ ]:
ff.f(mu=mu, dmu=0.9, delta=delta), ff.f(mu=mu, dmu=0.8, delta=delta), ff.f(mu=mu, dmu=1.2, delta=delta)

In [ ]:
ff.get_densities(mu=mu, dmu=.9)

## Check how $g$ changes with $d\mu$

$$
\frac{1}{g} = -\frac{1}{2}\int \frac{\d{k}}{2\pi}\frac{1}{E_k}\bigl(f(\omega_-)-f(\omega_+)\bigr).
$$
* So g would not change until dmu exceeds the gap, where the occupancy change

In [ ]:
dmus = np.linspace(-1.2 * delta, 1.2 * delta, 40)
gs = [ff.get_g(mu=mu, dmu=dmu, delta=delta, k_c=50) for dmu in dmus]
plt.plot(dmus, gs,'--')
plt.axvline(delta)
plt.axvline(-delta)

In [ ]:
dmus = np.linspace(-1.2 * delta, 1.2 * delta, 40)
gs = [ff.get_g(mu=mu * 1.2, dmu=dmu * 1.2, delta=delta, k_c=50) for dmu in dmus]
plt.plot(dmus, gs,'--')
plt.axvline(delta)
plt.axvline(-delta)

## Check the range of $d\mu$ that yields superfluid states

In [ ]:
dmus = np.linspace(-mu,mu,30)
states = [ff.check_superfluidity(mu=mu, dmu=dmu) for dmu in dmus]
plt.plot(dmus, states)

In [ ]:
dmu = 1.5
na, nb = ff.get_densities(mu=mu, dmu=dmu)
na, nb=na.n, nb.n
print(na, nb)
scale = 0.01
ks = np.linspace(-scale,scale, 100)
r = na/nb
dns = []
js = []
for k in ks:
    q, dq = (1 + r)*k, (1 - r)*k
    na_, nb_ = ff.get_densities(mu=mu, dmu=dmu, q=q, dq=dq)
    na_, nb_ = na_.n, nb_.n
    dn = np.sqrt((na - na_)**2 + (nb - nb_)**2)
    j = na_ * (q + dq) + nb_ * (q-dq)
    dns.append(dn)
    js.append(j)
plt.plot(ks, dns)
plt.plot(ks, js)
plt.axvline(0)

In [ ]:
%pylab inline --no-import-all
from ipywidgets import interact

def f(E, T):
    """Fermi distribution function"""
    T = max(T, 1e-12)
    return 1./(1+np.exp(E/T))


@interact(delta=(0, 1, 0.1), 
          mu_eF=(0, 2, 0.1),
          dmu=(-0.4, 0.4, 0.01),
          T=(0, 0.1, 0.01))
def go(delta=0.1, mu_eF=1.0, dmu=0.0, T=0.02):
    k = np.linspace(0, 1.4, 100)
    hbar = m = kF = 1.0
    eF = (hbar*kF)**2/2/m
    mu = mu_eF*eF
    #dmu = dmu_delta*delta
    mu_a, mu_b = mu + dmu, mu - dmu
    e_a, e_b = (hbar*k)**2/2/m - mu_a, (hbar*k)**2/2/m - mu_b
    e_p, e_m = (e_a + e_b)/2, (e_a - e_b)/2
    E = np.sqrt(e_p**2+abs(delta)**2)
    w_p, w_m = e_m + E, e_m - E
    
    # Occupation numbers
    f_p = 1 - e_p/E*(f(w_m, T) - f(w_p, T))
    f_m = f(w_p, T) - f(-w_m, T)
    f_a, f_b = (f_p+f_m)/2, (f_p-f_m)/2
    f_nu = f(w_m, T) - f(w_p, T)
    plt.subplot(211);plt.grid()
    plt.plot(k/kF, f_a, label='a')
    plt.plot(k/kF, f_b, label='b')
    plt.plot(k/kF, f_nu, label=r'$\nu$');plt.legend()
    plt.ylabel('n')
    plt.subplot(212);plt.grid()
    plt.plot(k/kF, w_p/eF, k/kF, w_m/eF)
    plt.xlabel('$k/k_F$')
    plt.ylabel(r'$\omega_{\pm}/\epsilon_F$')
    plt.axhline(0, c='y')

# Brutal Method